In [70]:
import pandas as pd
from rdkit import Chem
# discussion of circular fingerprints: https://pubs.acs.org/doi/10.1021/ci100050t
from rdkit.Chem import AllChem
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import os
import pickle
import pandas as pd

from sklearn.model_selection import train_test_split

#Load the Models:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier


#Load the Metrics:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

#Other fingerprint types to explore? 
#useful example: https://medium.com/@gurkamaldeol/predicting-environmental-carcinogens-with-logistic-regression-knn-gradient-boosting-and-7973f88eb8b3

In [15]:
# pull in the datasets:
training_datasets=[i for i in os.listdir('data_split_cleaned') if 'train' in i]
validation_datasets=[i for i in os.listdir('data_split_cleaned') if 'validate' in i]

In [21]:
data_map={
    'HIV': {'target':'HIV_active','structure':'smiles'},
    'bace':{'target':'active','structure':'mol'},
    'tox21':{'target':'NR-AhR','structure':'smiles'},
    'clintox':{'target':'CT_TOX','structure':'smiles'},
    'sol_del':{'target':'binned_sol','structure':'smiles'},
    'deepchem_Lipophilicity':{'target':'drug_like','structure':'smiles'}   
}
model_save_path='Simple_Models'

In [9]:
def generate_fingerprint(smiles,radius,bits):
    try:
        mol=Chem.MolFromSmiles(smiles)
        fp=AllChem.GetMorganFingerprintAsBitVect(mol,radius,bits)
        return(np.array(fp))
    except:
        print(f'{smiles} failed in RDkit')
        return (np.nan)

In [87]:
# Load the scoring and model functions:
scoring = ['accuracy', 'f1','roc_auc','neg_log_loss']

models={'Logistic_Regression':LogisticRegression(random_state=0,solver='lbfgs',max_iter=1000,verbose=False),
        'Random_Forest':RandomForestClassifier(random_state=0,n_jobs=-1),
        'KNN': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
        'Gradient_Boosted_Tree': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0),
        'SVM':SVC(C=1.0, kernel='linear', degree=3, gamma='scale',probability=True),
        'Dummy_Most_Frequent':DummyClassifier(strategy="most_frequent")
       }

In [88]:
# load the data set
# create finger prints on molecules
# extract X and Y for training
# Train the model on this data (cross val)
# save the scores to csv in folder
# save the model as a pickle 
# predict with the model using one trained on all. 
#save those scores to csv for datafile (train and val) 

In [89]:
def save_model(model,model_name):
    model_out=os.path.join('Simple_Models',model_name)

    if not os.path.exists(model_out):
        os.makedirs(model_out)
    with open(f'{model_name}_model.pkl','wb') as f:
        pickle.dump(clf,f)
    print(f'saved_{model_name}_to_{model_out}')

In [90]:
def featurize_dataset(dataset,radius,bits):
    df=pd.read_csv(os.path.join('data_split_cleaned',dataset))
    data_lookup=dataset.split('-')[0]
    target=data_map[data_lookup]['target']
    smiles=data_map[data_lookup]['structure']
    df['fp']=df[smiles].apply(lambda x: generate_fingerprint(x,radius,bits))
    df.dropna(subset=['fp',target],inplace=True) # Remove any failed fingerprints or missing targets
    X=df['fp'].to_list()
    y=df[target].to_list()
    return(X,y)

In [95]:
# Train the models:
for dataset in training_datasets:
    #extract the dataset name
    data_set_name=dataset.replace('.csv','').split('-')
    data_set_name=data_set_name[0]+'-'+data_set_name[1]
    
    X,y=featurize_dataset(dataset,2,1024) # featurize the dataset
    
    model_out=os.path.join('Simple_Models',data_set_name)
    if not os.path.exists(model_out):# make an output path
        os.makedirs(model_out)
    
    for model in tqdm(models):
        clf=models[model]
        # run cross val to estimate error:
        cv_result=cross_validate(clf , X, y, scoring=scoring, cv=5, return_estimator=False)
        pd.DataFrame(cv_result).describe().to_csv(os.path.join(model_out,model+'.csv'))
        
        clf.fit(X,y) # fit the model on the whole dataset:
        
        with open(os.path.join(model_out,f'{model}_model.pkl'),'wb') as f:
            pickle.dump(clf,f)
        print(f'saved_{model}_to_{model_out}')

 17%|███████▌                                     | 1/6 [00:00<00:00,  6.42it/s]

saved_Logistic_Regression_to_Simple_Models/sol_del-cluster


 33%|███████████████                              | 2/6 [00:02<00:05,  1.48s/it]

saved_Random_Forest_to_Simple_Models/sol_del-cluster


/opt/anaconda3/envs/Capstone/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/envs/Capstone/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no lo

...

saved_KNN_to_Simple_Models/bace-random


 67%|██████████████████████████████               | 4/6 [00:06<00:03,  1.76s/it]

saved_Gradient_Boosted_Tree_to_Simple_Models/bace-random


100%|█████████████████████████████████████████████| 6/6 [00:10<00:00,  1.71s/it]

saved_SVM_to_Simple_Models/bace-random
saved_Dummy_Most_Frequent_to_Simple_Models/bace-random
